In [ ]:
%matplotlib inline
import math
import pywt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import HBox, FloatSlider, IntSlider
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import importlib
import toolbox
importlib.reload(toolbox)

In [ ]:
plt.rcParams["figure.figsize"] = [16,9]
plt.rcParams.update({'font.size': 20})

In [ ]:
from toolbox import *

# Merkmale der Wavelet Analyse

 * Welche Information werden durch Wavelets extrahiert?
 * Welche Information werden durch Wavelets zerstört?

# Fokus

 * Wavelet Analyse von 1D Signalen
     * Polynome
     * Mit Rauschen
     * Mit periodischen Elementen
     * Ausschliesslich Haar / DB Wavelets
 * Anwendung zur Rauschunempfindlichen Ableitung?
 * Vergleich FFT

# Anekdoten



In Dokumentation zu Implementierung wird oft nur von Detail und Approximation Koeffizienten gesprochen. Die Approximation sind der Tiefpass Output und die Detail dem Hochpass Output.

## Begriffe in der Praxis

 * 1 Stufige Wavelet Analyse
   * Detail und Approximation Koeffizienten
![DFT](https://upload.wikimedia.org/wikipedia/commons/4/45/Wavelets_-_DWT.png)
Quelle: Wikipedia

## Filter Einschwingverhalten
 * Filter brauchen Zeit zum Einschwingen
 * Was füttere ich rein?
   * 0er?
   * Konstanten?
   * Periodisierung?


![](https://pywavelets.readthedocs.io/en/latest/_images/plot_boundary_modes.png)
Quelle: https://pywavelets.readthedocs.io/

# Analyse von Polynomen

In [ ]:
fig, data = print_signals(data_pure, labels);

# Analyse mit Haar

In [ ]:
plot_wavelet_analysis(data_pure[:], labels[:], 'db1')

# Ableitung

In [ ]:
plot_differentation_analysis(data_pure[:], labels[:],  1)

In [ ]:
plot_wavelet_analysis(data_pure[1:], labels[1:], 'db2')

In [ ]:
plot_differentation_analysis(data_pure[1:], labels[1:],  2)

In [ ]:
plot_wavelet_analysis(data_pure[1:], labels[1:], 'db3')

# Differenzieren von verrauschten Signalen

 * Rauschen "explodiert" bei mehrfachem nummerischen Differenzieren
 * Könnten Wavlets das umgehen?

In [ ]:
print_signals(data_with_noise[1:], labels[1:]);

In [ ]:
plot_wavelet_analysis(data_with_noise[1:], labels[1:], 'db1')

In [ ]:
plot_differentation_analysis(data_with_noise[1:], labels[1:], 1)

In [ ]:
plot_dwt_multi_level(data_with_noise[1:], labels[1:], 'db1', plt.figure(), 3)

In [ ]:
fig = plt.figure(figsize=[16, 14])
plot_dwt_multi_level(data_with_noise[1:], labels[1:5], 'db2', fig, 5)

In [ ]:
plot_differentation_analysis(data_with_noise[1:], labels[1:], 2)

## Demo Time

# Polynome mit Sinus

# Polynome mit Rauschen

# Information von Wavlets

 * Lokal: Hohe Frequenzen / Ableitung
 * Global: "DC"